<a href="https://colab.research.google.com/github/macgyver121/my_project/blob/main/SQL_chinook_project/SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL Project using Chinook.db

In [10]:
# Imports
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

In [11]:
path = '/content/chinook.db'

In [12]:
con = sq3.Connection(path)

In [13]:
query = """
SELECT *
FROM artists;
"""
q = pd.read_sql(query, con)
print(q)

     ArtistId                                               Name
0           1                                              AC/DC
1           2                                             Accept
2           3                                          Aerosmith
3           4                                  Alanis Morissette
4           5                                    Alice In Chains
..        ...                                                ...
270       271   Mela Tenenbaum, Pro Musica Prague & Richard Kapp
271       272                             Emerson String Quartet
272       273  C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...
273       274                                      Nash Ensemble
274       275                              Philip Glass Ensemble

[275 rows x 2 columns]


In [14]:
# show list of table
try:
	
	# Making a connection between sqlite3
	# database and Python Program
	sqliteConnection = sq3.connect(path)
	
	# If sqlite3 makes a connection with python
	# program then it will print "Connected to SQLite"
	# Otherwise it will show errors
	print("Connected to SQLite")

	# Getting all tables from sqlite_master
	sql_query = """SELECT name FROM sqlite_master
	WHERE type='table';"""

	# Creating cursor object using connection object
	cursor = sqliteConnection.cursor()
	
	# executing our sql query
	cursor.execute(sql_query)
	print("List of tables\n")
	
	# printing all tables list
	print(cursor.fetchall())

except sq3.Error as error:
	print("Failed to execute the above query", error)
	
finally:

	# Inside Finally Block, If connection is
	# open, we need to close it
	if sqliteConnection:
		
		# using close() method, we will close
		# the connection
		sqliteConnection.close()
		
		# After closing connection object, we
		# will print "the sqlite connection is
		# closed"
		print("the sqlite connection is closed")


Connected to SQLite
List of tables

[('albums',), ('sqlite_sequence',), ('artists',), ('customers',), ('employees',), ('genres',), ('invoices',), ('invoice_items',), ('media_types',), ('playlists',), ('playlist_track',), ('tracks',), ('sqlite_stat1',)]
the sqlite connection is closed


## Question 1: Which city have the most Invoices?

In [15]:
query = """
SELECT billingcity,COUNT(billingcity) AS Invoice_Number
FROM invoices
GROUP BY billingcity
ORDER BY Invoice_Number DESC
LIMIT 8;
"""
q = pd.read_sql(query, con)
print(q)

     BillingCity  Invoice_Number
0      São Paulo              14
1         Prague              14
2          Paris              14
3  Mountain View              14
4         London              14
5         Berlin              14
6    Yellowknife               7
7       Winnipeg               7


## Question 2: Which country has the best customers?

In [16]:
query = """
SELECT billingcountry,SUM(total) AS InvoiceTotal
FROM invoices
GROUP BY billingcountry
ORDER BY InvoiceTotal DESC
LIMIT 5;
"""
q = pd.read_sql(query, con)
print(q)

  BillingCountry  InvoiceTotal
0            USA        523.06
1         Canada        303.96
2         France        195.10
3         Brazil        190.10
4        Germany        156.48


## Question 3: Who is the best customer?

In [17]:
query = """
SELECT C.customerid, C.firstname, C.lastname, SUM(I.total) AS total_spending
FROM customers C
JOIN invoices I 
ON C.customerid = I.customerid
GROUP BY (C.customerid)
ORDER BY total_spending DESC
LIMIT 3;
"""
q = pd.read_sql(query, con)
print(q)

   CustomerId FirstName    LastName  total_spending
0           6    Helena        Holý           49.62
1          26   Richard  Cunningham           47.62
2          57      Luis       Rojas           46.62


## Question 4: Use your query to return the email, first name, last name, and Genre of all Pop Music listeners.Return your list ordered alphabetically by email address starting with A.

In [18]:
query = """
SELECT DISTINCT email AS Email,firstname AS FirstName, lastname AS LastName, genres.name AS Name
FROM customers
JOIN invoices ON invoices.customerid = customers.customerid
JOIN invoice_items ON invoice_items.invoiceid = invoices.invoiceid
JOIN tracks ON tracks.trackid = invoice_items.trackid
JOIN genres ON genres.genreid = tracks.genreid
WHERE genres.name LIKE 'Pop'
ORDER BY email;
"""
q = pd.read_sql(query, con)
print(q)

                       Email  FirstName      LastName Name
0     astrid.gruber@apple.at     Astrid        Gruber  Pop
1   fernadaramos4@uol.com.br   Fernanda         Ramos  Pop
2   frantisekw@jetbrains.com  František   Wichterlová  Pop
3        jfernandes@yahoo.pt       João     Fernandes  Pop
4    johavanderberg@yahoo.nl   Johannes  Van der Berg  Pop
5     johngordon22@yahoo.com       John        Gordon  Pop
6        kachase@hotmail.com      Kathy         Chase  Pop
7      kara.nielsen@jubii.dk       Kara       Nielsen  Pop
8      leonekohler@surfeu.de     Leonie        Köhler  Pop
9       luisg@embraer.com.br       Luís     Gonçalves  Pop
10   marc.dubois@hotmail.com       Marc        Dubois  Pop
11      patrick.gray@aol.com    Patrick          Gray  Pop
12     steve.murray@yahoo.uk      Steve        Murray  Pop


## Question 5: Who is writing the pop music and how many?

In [19]:
query = """
SELECT artists.artistid, artists.name,COUNT(artists.artistid) AS number_of_songs
FROM tracks
JOIN albums ON albums.albumid = tracks.albumid
JOIN artists ON artists.artistid = albums.artistid
JOIN genres ON genres.genreid = tracks.genreid
WHERE genres.name LIKE 'Pop'
GROUP BY artists.artistid
ORDER BY number_of_songs DESC
LIMIT 10;
"""
q = pd.read_sql(query, con)
print(q)

   ArtistId             Name  number_of_songs
0       150               U2               23
1        21  Various Artists               14
2       252    Amy Winehouse               11


## Question 6: Return all the track names that have a song length longer than the average song length.

In [22]:
query = """
SELECT name,Milliseconds
FROM tracks
WHERE Milliseconds > (
	SELECT AVG(Milliseconds) AS avg_track_length
	FROM tracks)
ORDER BY Milliseconds DESC;
"""
q = pd.read_sql(query, con)
print(q)

                                                  Name  Milliseconds
0                               Occupation / Precipice       5286953
1                              Through a Looking Glass       5088838
2                          Greetings from Earth, Pt. 1       2960293
3                              The Man With Nine Lives       2956998
4                          Battlestar Galactica, Pt. 2       2956081
..                                                 ...           ...
489                                   22 Acacia Avenue        395572
490                                  The Unforgiven II        395520
491                                 The Shortest Straw        395389
492  Concerto for Clarinet in A Major, K. 622: II. ...        394482
493                                        Wicked Ways        393691

[494 rows x 2 columns]


## Question 7: Find all the customers who bought the best-selling singer's songs in descending order of purchase amount.

In [25]:
query = """
SELECT artists.artistid AS artist_id,artists.name AS artist_name, SUM(invoice_items.UnitPrice*invoice_items.Quantity) AS total_sales
FROM invoice_items
JOIN tracks ON tracks.trackid = invoice_items.trackid
JOIN albums ON albums.albumid = tracks.albumid
JOIN artists ON artists.artistid = albums.artistid
GROUP BY 1
ORDER BY 3 DESC
LIMIT 1
"""
q = pd.read_sql(query, con)
print(q)

   artist_id  artist_name  total_sales
0         90  Iron Maiden        138.6


In [29]:
query = """
WITH tbl_best_selling_artist AS(
    SELECT artists.artistid AS artist_id,artists.name AS artist_name, SUM(invoice_items.UnitPrice*invoice_items.Quantity) AS total_sales
    FROM invoice_items
    JOIN tracks ON tracks.trackid = invoice_items.trackid
    JOIN albums ON albums.albumid = tracks.albumid
    JOIN artists ON artists.artistid = albums.artistid
    GROUP BY 1
    ORDER BY 3 DESC
    LIMIT 1
)

SELECT bsa.artist_name,SUM(il.UnitPrice*il.Quantity) AS amount_spent,c.customerid,c.firstname,c.lastname
FROM invoices i
JOIN customers c ON c.customerid = i.customerid
JOIN invoice_items il ON il.invoiceid = i.invoiceid
JOIN tracks t ON t.trackid = il.trackid
JOIN albums alb ON alb.albumid = t.albumid
JOIN tbl_best_selling_artist bsa ON bsa.artist_id = alb.artistid
GROUP BY 1,3,4,5
ORDER BY 2 DESC;

"""
q = pd.read_sql(query, con)
print(q)

    artist_name  amount_spent  CustomerId  FirstName     LastName
0   Iron Maiden         17.82          55       Mark       Taylor
1   Iron Maiden         15.84          35   Madalena      Sampaio
2   Iron Maiden         13.86          16      Frank       Harris
3   Iron Maiden         13.86          36     Hannah    Schneider
4   Iron Maiden          8.91           5  František  Wichterlová
5   Iron Maiden          8.91          27    Patrick         Gray
6   Iron Maiden          5.94          19        Tim        Goyer
7   Iron Maiden          5.94          39    Camille      Bernard
8   Iron Maiden          5.94          58      Manoj       Pareek
9   Iron Maiden          4.95           7     Astrid       Gruber
10  Iron Maiden          4.95          25     Victor      Stevens
11  Iron Maiden          3.96          15   Jennifer     Peterson
12  Iron Maiden          3.96          54      Steve       Murray
13  Iron Maiden          2.97          50    Enrique        Muñoz
14  Iron M